In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer
from torch.optim import Adam
import torch.nn.functional as F
import pandas as pd
import numpy as np
import os
import sys

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#enter your file name for the attack
file_path_attack = '/content/drive/MyDrive/673Project/Datasets/IrrelevantSymptoms_DiseasesOutcomes.csv'

In [3]:
# Load ClinicalBERT
model_name = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
#load the dataset
df = pd.read_csv(file_path_attack, encoding='ISO-8859-1')
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
print(len(shuffled_df))

150


In [ ]:
#create list of diseases with and without attacks
diseases_attack = []
diseases_no_attack = []
chunk_size = 15

shuffled_df['Diseases_with_attack']
shuffled_df['Diseases_without_attack']
for i in range(0,len(shuffled_df)):
   diseases_with_attack = shuffled_df['Diseases_with_attack'][i].split(",")
   diseases_with_attack = diseases_with_attack[0:chunk_size]
   diseases_attack.extend(diseases_with_attack);
   diseases_without_attack = shuffled_df['Diseases_without_attack'][i].split(",")
   diseases_without_attack = diseases_without_attack[0:chunk_size]
   diseases_no_attack.extend(diseases_without_attack);


In [ ]:
print(len(diseases_attack))
print(len(diseases_no_attack))

750
750


In [ ]:
#the class for the disease datset. this can be used for both tokeninzing the diseases with an dwithout attack
class DiseasesDataset(Dataset):
    def __init__(self, disease_attack_text , tokenizer,max_length=128):
        self.disease_attack_texts = disease_attack_text
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.disease_attack_texts)

    def __getitem__(self, idx):
        text = self.disease_attack_texts[idx]
        # Tokenize text
        diseases_attack_token = self.tokenizer(
            self.disease_attack_texts[idx] ,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )


        return {
            "input_ids": diseases_attack_token["input_ids"].squeeze(0),
            "attention_mask": diseases_attack_token["attention_mask"].squeeze(0)
            #"label_index":torch.tensor(label, dtype=torch.long),
        }


dataset_attack_ds = DiseasesDataset(diseases_attack,  tokenizer)
dataloader_attack_disease = DataLoader(dataset_attack_ds, batch_size=32, shuffle=True)

dataset_no_attack_ds = DiseasesDataset(diseases_no_attack,  tokenizer)
dataloader_attack_no_disease = DataLoader(dataset_no_attack_ds, batch_size=32, shuffle=True)

In [ ]:
#clinical bert model just to get the embeddings , actually it does not have a classifier
class ClinicalBERTClassifier(nn.Module):
    def __init__(self, clinical_bert):
        super(ClinicalBERTClassifier, self).__init__()
        self.bert = clinical_bert

    def forward(self, input_ids, attention_mask):
        # Pass inputs through ClinicalBERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        #outputs = self.bert(input_ids=input_ids_attack, attention_mask=attention_mask_attack)
        return outputs



In [ ]:
model = ClinicalBERTClassifier(model)

In [ ]:
#embeddings to get the evaluation
def evaluate_model(model, dataloader):
    model.eval()
    embeddings_list = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]
            print(attention_mask)
            embeddings = model(input_ids, attention_mask)
            cls_output = embeddings.last_hidden_state[:, 0, :]
            embeddings_list.extend(cls_output.tolist())


    return embeddings_list

# get the embeddings for the diseases of attacked clinical note
embeddings_list_attack = evaluate_model(model, dataloader_attack_disease)


tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1

In [ ]:
print(len(embeddings_list_attack))

2250


In [ ]:
disease_embedding_per_attack = [embeddings_list_attack[i:i + chunk_size] for i in range(0, len(embeddings_list_attack), chunk_size)]
print(len(disease_embedding_per_attack))

150


In [ ]:
#get the embeddings for diseases of clinical notes without attack
embeddings_list_no_attack = evaluate_model(model, dataloader_attack_no_disease)
disease_embedding_per_no_attack = [embeddings_list_no_attack[i:i + chunk_size] for i in range(0, len(embeddings_list_no_attack), chunk_size)]


tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1

In [ ]:
print(len(disease_embedding_per_no_attack))

150


In [ ]:
#specify the range to evaluate top 5, top 10, top 7, top15
top_k = 15

In [ ]:
count_diseases_present = []
for i in range (0,len(disease_embedding_per_no_attack)):
   #print(len(disease_embedding_per_no_attack[i][0]))
   #sys.exit()

   no_attack = np.array(disease_embedding_per_no_attack[i])
   row_norms = np.linalg.norm(no_attack, axis=1, keepdims=True)  # Normalize along rows
   normalized_rows_no_attack = no_attack / row_norms
   #print(i)
   attack = np.array(disease_embedding_per_attack[i])
   row_norms = np.linalg.norm(attack, axis=1, keepdims=True)  # Normalize along rows
   normalized_rows_attack = attack / row_norms
   no_diseases_present_count = 0
   C = np.dot(normalized_rows_no_attack, np.transpose(normalized_rows_attack))
   for j in range(0,top_k):
      for k in range (0,top_k):
         if C[j][k] >=0.95:
            no_diseases_present_count = no_diseases_present_count+1;
            break;

   count_diseases_present.append(top_k - no_diseases_present_count)

#this prints the mean difference for a particular top k range
print(np.mean(count_diseases_present))

1.1866666666666668


In [ ]:
print(C)

[[202.55294007 202.14388057 209.1336043  204.91642419 205.05928052
  211.54053657 204.04933724 207.94611429 209.09730028 204.97377315
  201.78954721 205.69643294 201.65614826 208.05590146 204.37780422]
 [198.55847174 198.15747915 205.00936123 200.87534651 201.01538562
  207.36882732 200.02535904 203.84528925 204.97377315 200.9315645
  197.81013346 201.639973   197.67936522 203.95291136 200.34734846]
 [204.18766508 203.77530422 210.82143926 206.57022395 206.71423321
  213.24779693 205.69613908 209.62436548 210.78484224 206.62803575
  203.41811117 207.35652784 203.28363562 209.73503871 206.02725699]
 [202.55294007 202.14388057 209.1336043  204.91642419 205.05928052
  211.54053657 204.04933724 207.94611429 209.09730028 204.97377315
  201.78954721 205.69643294 201.65614826 208.05590146 204.37780422]
 [195.39618231 195.00157602 201.7443334  197.67615798 197.81396679
  204.06622207 196.83970761 200.59880071 201.7093121  197.73148063
  194.65976224 198.42860685 194.53107665 200.7047088  197.1